### 1 . 创建文件结构

In [1]:
import os

def create_sub_folders(base_path):
    """在指定路径下创建csv文件夹, shp文件夹, shp2tif文件夹, tif文件夹"""
    sub_folders = ['csv']
    for sub_folder in sub_folders:
        sub_folder_path = os.path.join(base_path, sub_folder)
        if not os.path.exists(sub_folder_path):
            os.makedirs(sub_folder_path)

def create_folder_structure(root_dir):
    # 定义文件夹结构
    folder_structure = {
        "建设用地": ["工业用地", "商业用地", "公服用地", "住宅用地"],
        "农用地": ["林地","园地", "草地", "设施农用地", "坑塘水面", "耕地"]
    }
    
    # 创建根目录
    if not os.path.exists(root_dir):
        os.makedirs(root_dir)
    
    # 创建子文件夹
    for main_folder, sub_folders in folder_structure.items():
        main_folder_path = os.path.join(root_dir, main_folder)
        if not os.path.exists(main_folder_path):
            os.makedirs(main_folder_path)
        
        for sub_folder in sub_folders:
            sub_folder_path = os.path.join(main_folder_path, sub_folder)
            if not os.path.exists(sub_folder_path):
                os.makedirs(sub_folder_path)
            
            # 在每个子文件夹下创建csv文件夹, shp文件夹, shp2tif文件夹, tif文件夹
            create_sub_folders(sub_folder_path)
            
            # 创建一个名为 sub_folder 的shp文件
            # shp_file_path = os.path.join(sub_folder_path, f"{sub_folder}.shp")
            # with open(shp_file_path, 'w') as shp_file:
            #     shp_file.write('')  # 创建一个空文件
    print(root_directory,"文件结构创建完成。")
    
def create_folders(base_path):
    folders = [
        "shp",
        "shp2tif",
        "tif"]
    
    for folder in folders:
        path = os.path.join(base_path, folder)
        os.makedirs(path, exist_ok=True)
        print(f"创建文件夹: {path}")


- 在root_directory指定创建的路径，有建设用地和农用地文件夹下共有10个小类文件夹
- 在base_path 指定创建的路径，有shp shp2tif tif

In [2]:
# 指定根目录路径
root_directory = "data" 
create_folder_structure(root_directory)

data 文件结构创建完成。


In [3]:
# # 指定文件夹创建的位置
base_path = "pri_data"  # 请修改为你想要创建文件夹的位置
create_folders(base_path)

创建文件夹: pri_data\shp
创建文件夹: pri_data\shp2tif
创建文件夹: pri_data\tif


### 2 . 移动文件到指定的文件夹

- csv 机器学习数据集 需要数据处理完全后生成
- shp 待进行近邻分析的shp文件，点或者折线要素
- shp2tif 待进行转化为核密度栅格图的SHP文件

### 3 . 编码与反编码

In [5]:
import os
import json

def parse_txt_to_json(txt_file_path, json_file_path):
    with open(txt_file_path, 'r', encoding='utf-8') as file:
        lines = file.readlines()
    
    # Initialize a dictionary to store the file pairs
    file_pairs = {}

    for line in lines:
        # Remove the newline character and split the line by " -> "
        old_path, new_path = line.strip().split(" -> ")
        # Get the file names without path
        old_name = os.path.basename(old_path)
        new_name = os.path.basename(new_path)
        # Get the base names without extensions
        old_base = os.path.splitext(old_name)[0]
        new_base = os.path.splitext(new_name)[0]
        # Store the pair in the dictionary
        file_pairs[old_base] = new_base

    # Save the dictionary to a JSON file
    with open(json_file_path, 'w', encoding='utf-8') as json_file:
        json.dump(file_pairs, json_file, ensure_ascii=False, indent=4)

if __name__ == "__main__":
    txt_file_path = './shp2tif_random.txt'  # Update this path to your txt file
    json_file_path = './shp2tif.json'    # Update this path to your desired json file
    parse_txt_to_json(txt_file_path, json_file_path)
    print(f'TXT content has been converted to JSON and saved to {json_file_path}')


TXT content has been converted to JSON and saved to ./shp2tif.json


In [6]:
import csv
import json

# 读取映射字典
def load_mapping(mapping_file):
    with open(mapping_file, 'r', encoding='utf-8') as file:
        return json.load(file)

# 去除指定的前缀和后缀
def strip_suffix_prefix(column_name, suffixes, prefixes):
    for suffix in suffixes:
        if column_name.endswith(suffix):
            column_name = column_name[:-len(suffix)]
    for prefix in prefixes:
        if column_name.startswith(prefix):
            column_name = column_name[len(prefix):]
    return column_name

# 更新CSV文件的列名称
def update_csv_column_names(csv_file, mapping, output_file, suffixes=None, prefixes=None):
    suffixes = suffixes or []
    prefixes = prefixes or []
    
    #reverse_mapping = {v: k for k, v in mapping.items()}
    reverse_mapping = mapping

    with open(csv_file, 'r', encoding='gbk') as infile, open(output_file, 'w', encoding='utf-8-sig', newline='') as outfile:
        reader = csv.DictReader(infile)
        fieldnames = reader.fieldnames
        new_fieldnames = []

        
        for field in fieldnames:
            aaa=''
            if field.endswith('_dis'):aaa='_dis'
            stripped_name = strip_suffix_prefix(field, suffixes, prefixes)
            new_fieldname = reverse_mapping.get(stripped_name, field)+aaa
            new_fieldnames.append(new_fieldname)
            
            print(stripped_name,new_fieldname)

        writer = csv.DictWriter(outfile, fieldnames=new_fieldnames)
        writer.writeheader()
        for row in reader:
            new_row = {new_fieldname: row[old_fieldname] for old_fieldname, new_fieldname in zip(fieldnames, new_fieldnames)}
            writer.writerow(new_row)

# 主函数
def main():
    csv_file = 'pri_datasets\草地.csv'  # 修改为输入的CSV文件路径
    mapping_file = r'C:\Users\黄国斌\Desktop\模块化\all.json'  # 修改为映射字典文件路径
    output_file = './temp/草地.csv'  # 修改为输出的CSV文件路径
    prefixes = ['']
    suffixes = ['_dis']
    mapping = load_mapping(mapping_file)
    update_csv_column_names(csv_file, mapping, output_file, suffixes, prefixes)
    print(f"Updated CSV saved to {output_file}")

if __name__ == "__main__":
    main()

pointID pointID
Y Y
clas clas
0J2Eei poi_140102_dis
0lNgn9 poi_140300_dis
0lqILL poi_190111_dis
0lRuxs poi_120100_dis
0q9zMY poi_130105_dis
16s2nz poi_210209_dis
1EMEJI poi_180109_dis
1FGMod poi_230109_dis
1JuBfd poi_190401_dis
1LjRDl poi_1402_dis
1XF1X0 旅游景区点_dis
29KMnS poi_210105_dis
2lyMuJ 南宁市现状公园绿地（已建成）_dis
2r4Hh0 poi_190500_dis
2rzl86 poi_130104_dis
2UAhV5 定修段_dis
317Ntu poi_160107_dis
32XMZ5 poi_200200_dis
37ml0o poi_210400_dis
3AqzlF poi_190113_dis
3PzlFt poi_120200_dis
3RIUiC poi_170106_dis
4fvVIy poi_1302_dis
4G9UYJ poi_1903_dis
4R2Rn7 poi_2002_dis
589fg9 现状110k伏埋地电缆_dis
58Tx9G 供水路线_dis
5A0TWR poi_130500_dis
5gKx0K poi_180106_dis
5kEZ8d poi_210302_dis
5MOvkn poi_130806_dis
5nBfi6 poi_2202_dis
5RjxqC poi_11_dis
5ToHm9 公交场站_dis
68jgTI poi_130807_dis
6AwPzR poi_130410_dis
6CZIhx poi_260116_dis
6dqoNu RD-02-快速路_dis
6ftQpM poi_130801_dis
6IBlpO poi_18_dis
6PNP8e poi_210212_dis
78CE4w poi_1904_dis
7jHpTb poi_160108_dis
7l7BuB poi_130502_dis
7Nc2i0 poi_170201_dis
7nkLA1 poi_1101_dis


### poi 代码表

In [17]:
import json

def generate_json_from_paragraph(paragraph, json_file):
    num_to_desc = {}
    lines = paragraph.strip().split('\n')
    for line in lines:
        parts = line.strip().split()
        if len(parts) >= 2:
            num = parts[1]
            desc = parts[0].strip()
            num_to_desc[num] = desc
    
    # 打印生成的JSON内容
    print('生成的JSON内容：')
    print(json.dumps(num_to_desc, ensure_ascii=False, indent=4))
    
    # 写入JSON文件
    with open(json_file, 'w', encoding='utf-8') as f:
        json.dump(num_to_desc, f, ensure_ascii=False, indent=4)
    
    print(f'已生成JSON文件：{json_file}')

# 使用示例
paragraph = """
餐馆 	110100 
中餐馆 	110101 
异国风味 	110102 
地方风味与名店 	110103 
快餐 	110200 
休闲餐饮 	110300 
酒吧 	110301 
冷饮店 	110302 
咖啡馆 	110303 
茶楼、茶艺馆 	110304 
商业性住宿 	120100 
星级宾馆 	120101 
旅馆、招待所 	120102 
酒店式公寓 	120103 
普通出租公寓 	120104 
居民住宿 	120200 
小区 	120201 
住宅楼 	120202 
综合零售 	130100 
小商品城 	130101 
百货商城、百货商店 	130102 
日杂店 	130103 
便利店 	130104 
超市 	130105 
烟酒副食、地方特产 	130200 
面包房 	130201 
农贸市场 	130202 
茶叶、茶具专卖店 	130203 
香烟/酒类专卖店 	130204 
地方土特产专卖店 	130205 
保健品专卖店 	130206 
副食专卖店 	130207 
母婴用品、儿童用品零售 	130300 
服装、鞋帽店 	130301 
眼镜店 	130302 
母婴用品、儿童用品 	130303 
化妆品专卖店 	130304 
文化、体育用品及器材零售 	130400 
体育用品店 	130401 
书店 	130402 
报刊亭 	130403 
音像店 	130404 
钟表，照相机专卖店 	130405 
珠宝首饰店 	130406 
工艺礼品 	130407 
古玩字画店 	130408 
乐器专卖店 	130409 
玩具店 	130410 
文化用品店 	130411 
医药及医疗器材零售 	130500 
药店 	130501 
医疗器械 	130502 
家用电器及电子产品零售 	130600 
电器零售 	130601 
电脑专卖店 	130602 
通信器材 	130603 
五金、家具及室内装修材料零售 	130700 
五金、机电、仪器仪表 	130701 
家具 	130702 
家装建材零售 	130703 
灯饰 	130704 
家居用品 	130705 
其他零售 	130800 
花卉专卖店 	130801 
宠物及宠物用品专卖店 	130802 
旧货市场 	130803 
自行车、电动车、摩托车销售 	130804 
丧葬用品 	130805 
宗教用品 	130806 
成人用品零售 	130807 
汽车及相关产品零售 	140100 
厂家推荐汽车零售 	140101 
汽车用品、汽车养护 	140102 
汽车零配件零售 	140103 
汽车维护与保养 	140200 
厂家推荐汽车修理 	140201 
机动车检测场 	140202 
汽车销售及服务 	140300 
厂家推荐车零售及修理 	140301 
汽车销售、汽车服务 	140302 
厂家推荐二手车交易 	140303 
二手车交易场所 	140304 
金融服务 	150100 
代表 	150101 
ATM/自助银行 	150102 
证券公司｜营业部代表 	150103 
典当拍卖、贷款担保 	150104 
保险业 	150200 
教育 	160100 
幼儿园/托儿所 	160101 
小学 	160102 
中学 	160103 
中专/职高/技校 	160104 
高等院校 	160105 
教学楼 	160106 
院/系 	160107 
学校报名处 	160108 
驾校 	160109 
考试中心 	160110 
党校 	160111 
高教园区 	160112 
文化、媒体 	160200 
报社、出版社、杂志社 	160201 
电台、电视台、电影制片厂 	160202 
图书馆 	160203 
资料馆、档案馆 	160204 
博物馆、纪念馆、展览馆、陈列馆 	160205 
美术馆 	160206 
科技馆 	160207 
文化馆、活动中心 	160208 
文工团、歌舞团、艺术团 	160209 
医疗机构 	170100 
综合医院 	170101 
专科医院 	170102 
私人诊所 	170103 
医院内部医疗设施 	170104 
急诊 	170105 
社区医疗 	170106 
牙科诊所 	170107 
防疫站 	170108 
体检机构 	170109 
献血屋 	170110 
社会保障机构 	170200 
福利院、敬老院 	170201 
运动场馆 	180100 
羽毛球场 	180101 
网球场 	180102 
保龄球馆 	180103 
滑雪场 	180104 
高尔夫球场 	180105 
高尔夫练习场 	180106 
赛马场 	180107 
垂钓 	180109 
溜冰场 	180110 
游泳馆 	180111 
休闲娱乐 	180200 
练歌房、KTV 	180201 
夜总会、歌舞厅、迪厅 	180202 
娱乐、健身 	180203 
棋牌室 	180204 
网吧 	180205 
赛马会 	180206 
博彩 	180207 
彩票销售 	180208 
电影院 	180209 
剧场、戏院、音乐厅 	180210 
赛狗场 	180211 
休闲度假 	180300 
农家乐/民俗游 	180301 
度假村、疗养院 	180302 
天然浴场 	180303 
公园 	180304 
街心公园 	180305 
广场 	180306 
游乐园 	180307 
动物园 	180308 
植物园 	180309 
水族馆 	180310 
风景名胜 	180400 
观光游览车站点 	180401 
景区服务点 	180402 
政府及管理机构 	190100 
中央人民政府 	190101 
省/直辖市/自治区政府 	190102 
地级市政府 	190103 
区县政府 	190104 
乡、镇政府 	190105 
村委会/居委会 	190106 
国家级政府机关 	190107 
省级政府机关 	190108 
地级市级政府机关 	190109 
区县级政府机关 	190110 
乡、镇级政府机关 	190111 
驻京、驻地方办事处 	190112 
行政办公大厅 	190113 
边检口岸 	190114 
公检法机构 	190200 
公安局 	190201 
派出所 	190202 
交警队 	190203 
消防 	190204 
法院 	190205 
检察院 	190206 
海关 	190207 
社会团体、协会 	190300 
协会 	190301 
行业性团体 	190302 
宗教 	190400 
寺庙、道观 	190401 
天主教 	190402 
基督教 	190403 
伊斯兰教 	190404 
其他外国办事机构 	190500 
外国大使馆/领事馆 	190501 
签证处 	190502 
商业设施 	200100 
会议中心、展览中心 	200101 
培训中心 	200102 
大厦 	200103 
商务中心/会馆 	200104 
商业综合体 	200105 
其他租赁服务 	200200 
汽车租赁 	200201 
法律、商务服务 	200300 
事务所 	200301 
商务服务 	200302 
人才市场 	200303 
旅行社 	200304 
居家服务 	210100 
家政服务 	210101 
摩托车、自行车、电动车维护与保养 	210102 
家电维修 	210103 
送水站 	210104 
宠物医院 	210105 
个人服务 	210200 
洗衣店/干洗店 	210201 
服装定制 	210202 
皮草/鞋/包保养 	210203 
美容院、理发店 	210204 
美甲、纹身 	210205 
洗浴中心、SPA、足浴 	210206 
婚庆服务 	210207 
火葬场、殡仪馆 	210208 
陵园、公墓 	210209 
邮局 	210210 
照相、影楼、彩扩 	210211 
公共厕所 	210212 
房产中介 	210213 
办公服务 	210214 
货物自提点 	210215 
婚姻介绍所 	210216 
票、费服务 	210300 
电信 	210301 
票务中心｜定票处 	210302 
缴费营业厅 	210303 
风景名胜售票点 	210304 
其他服务 	210400 
公司 	220100 
厂矿企业 	220200 
工业园 	220300 
其他单位 	220400 
客货运输 	230100 
火车站 	230101 
火车站出发到达 	230102 
货运火车站 	230103 
客运汽车站 	230104 
地铁站出入口 	230105 
地铁站主点 	230106 
磁悬浮出入口 	230107 
港口、码头 	230108 
机场 	230109 
机场附属设施 	230110 
物流、快运 	230111 
其他交通设施 	230200 
桥 	230201 
立交桥 	230202 
高速出口 	230203 
高速入口 	230204 
高速服务区 	230205 
高速停车区 	230206 
收费站 	230207 
室内停车场 	230208 
地上露天停车场 	230209 
公交换乘停车场 	230210 
货车停车场 	230211 
加油站 	230212 
加气站 	230213 
加油加气站 	230214 
电动汽车充电站 	230215 
超限超载检测站 	230216 
刹车失灵缓冲区 	230217 
刹车冷却区 	230218 
占道停车场 	230219 
加水站 	230220 
紧急停车带 	230221 
地上非露天停车场 	230222 
地下停车场 	230223 
露营房车营地 	230224 
普通道路服务区/停车区 	230225 
环岛名 	230226 
隧道 	230227 
枢纽 	230228 
互通 	230229 
道路名 	230230 
科研机构 	240100 
农林牧渔生产 	250100 
农林牧渔服务 	250200 
行政地名 	260100 
大洲名 	260101 
国家名 	260102 
首都名 	260103 
省级地名 	260104 
省会名 	260105 
直辖市级地名 	260106 
地级市地名 	260107 
区县级地名 	260108 
乡镇级地名 	260109 
街道级地名 	260110 
行政村 	260111 
自然村 	260112 
村屯级地名 	260113 
村组级地名 	260114 
社区 	260115 
集镇 	260116 
区片名 	260117 
自然地物 	260200 
海湾海峡 	260201 
岛屿 	260202 
山 	260203 
河流 	260204 
湖泊 	260205 
山谷、峡谷 	260206 
河口 	260207 
海岛 	260208 
山洞 	260209 
泉 	260210 
江心洲 	260211 
门牌信息 	260300 
地名门牌 	260301 
道路门牌 	260302 
小区门牌 	260303 
居民楼门牌 	260304 
标志性建筑物 	260400 
人工地物 	260500 
阁楼 	260501 
井 	260502 
石刻 	260503 



餐饮 11
住、宿  12
批发、零售 13
汽车销售及服务 14
金融、保险 15
教育、文化 16
卫生、社保 17
运动、休闲 18
公共设施 19
商业设施、商务服务 20
居民服务 21 
公司企业 22
交通运输、仓储  23
科研及技术服务   24
农林牧渔业 25
地名地址 26


餐馆 1101
快餐 1102
休闲餐饮 1103
商业性住宿 1201
居民住宿 1202
综合零售 1301
饮料及烟酒制品零售 1302
母婴用品、儿童用品零售 1303
文化、体育用品及器材零售  1304
医药及医疗器材零售  1305
家用电器及电子产品零售 1306
五金、家具及室内装修材料零售  1307
其他零售 1308
汽车及相关产品零售 1401
汽车维护与保养  1402
汽车销售及服务 1403
金融服务 1501
保险业 1502
教育 1601
文化、媒体  1602
医疗机构 1701
社会保障机构  1702
运动场馆 1801
休闲娱乐 1802
休闲度假 1803
风景名胜  1804
政府及管理机构  1901
公检法机构  1902
社会团体、协会 1903
宗教 1904
国际组织  1905
商业设施  2001
租赁服务 2002
法律、商务服务 2003
居家服务 2101
个人服务 2102
票、费服务 2103
其他服务 2104 
公司 2201
厂矿企业 2202
工业园 2203
其他单位 2204
客货运输 2301
交通附属设施 2302
科研机构 2401
农林牧渔生产  2501
农林牧渔服务 2502
行政地名 2601
自然地物 2602
门牌信息  2603
标志性建筑物 2604
人工地物 2605
"""
generate_json_from_paragraph(paragraph, './all_mapping.json')


生成的JSON内容：
{
    "110100": "餐馆",
    "110101": "中餐馆",
    "110102": "异国风味",
    "110103": "地方风味与名店",
    "110200": "快餐",
    "110300": "休闲餐饮",
    "110301": "酒吧",
    "110302": "冷饮店",
    "110303": "咖啡馆",
    "110304": "茶楼、茶艺馆",
    "120100": "商业性住宿",
    "120101": "星级宾馆",
    "120102": "旅馆、招待所",
    "120103": "酒店式公寓",
    "120104": "普通出租公寓",
    "120200": "居民住宿",
    "120201": "小区",
    "120202": "住宅楼",
    "130100": "综合零售",
    "130101": "小商品城",
    "130102": "百货商城、百货商店",
    "130103": "日杂店",
    "130104": "便利店",
    "130105": "超市",
    "130200": "烟酒副食、地方特产",
    "130201": "面包房",
    "130202": "农贸市场",
    "130203": "茶叶、茶具专卖店",
    "130204": "香烟/酒类专卖店",
    "130205": "地方土特产专卖店",
    "130206": "保健品专卖店",
    "130207": "副食专卖店",
    "130300": "母婴用品、儿童用品零售",
    "130301": "服装、鞋帽店",
    "130302": "眼镜店",
    "130303": "母婴用品、儿童用品",
    "130304": "化妆品专卖店",
    "130400": "文化、体育用品及器材零售",
    "130401": "体育用品店",
    "130402": "书店",
    "130403": "报刊亭",
    "130404": "音像店",
    "130405": "钟表，照相机专卖店

In [16]:
import csv
import json
import re

# 加载JSON文件
with open('mapping.json', 'r', encoding='utf-8') as f:
    mapping = json.load(f)

def replace_and_print_column_names(filename, mapping):
    with open(filename, 'r', newline='', encoding='utf-8') as csvfile:
        reader = csv.reader(csvfile)
        column_names = next(reader)

        # 使用正则表达式匹配各种格式的列名
        pattern_six_digits = re.compile(r'poi_(\d{6})_dis')
        pattern_two_digits = re.compile(r'poi_(\d{2})_dis')
        pattern_four_digits = re.compile(r'poi_(\d{4})_dis')
        pattern_generic_dis = re.compile(r'(.*)_dis$')

        new_column_names = []
        non_matching_columns = {
            'two_digits': [],
            'four_digits': [],
            'other': []
        }
        modified_six_digits = []
        modified_generic_dis = []
        renamed_others = []

        rename_mapping = {
            'RKMD': '人口密度',
            'DGSJ': '灯光数据'
        }

        for col in column_names:
            match_six = pattern_six_digits.match(col)
            match_two = pattern_two_digits.match(col)
            match_four = pattern_four_digits.match(col)
            match_generic = pattern_generic_dis.match(col)
            
            if match_six:
                key = match_six.group(1)
                if key in mapping:
                    new_col_name = f"距{mapping[key]}的距离"
                    new_column_names.append(new_col_name)
                    modified_six_digits.append(new_col_name)
                else:
                    new_column_names.append(col)
                    non_matching_columns['other'].append(col)
            elif match_two:
                non_matching_columns['two_digits'].append(col)
            elif match_four:
                non_matching_columns['four_digits'].append(col)
            elif match_generic:
                new_col_name = f"距{match_generic.group(1)}的距离"
                new_column_names.append(new_col_name)
                modified_generic_dis.append(new_col_name)
            elif col in rename_mapping:
                new_col_name = rename_mapping[col]
                new_column_names.append(new_col_name)
                renamed_others.append(new_col_name)
            else:
                new_column_names.append(col)
                non_matching_columns['other'].append(col)
        
        # # 打印修改后的列名称
        # print("修改后的列名称：")
        # print(new_column_names)
        
        # 打印poi_六位数字_dis的修改列名称
        print("poi_六位数字_dis的修改后列名称如下：")
        print(modified_six_digits)
        print("\n\n\n")

        # 打印形如xxx_dis的修改列名称
        print("形如xxx_dis，如南宁市现状公园绿地（已建成）_dis的修改列名称：")
        print(modified_generic_dis)
        print("\n\n\n")
        
        # 打印不匹配的列名称
        print("两位数字形式的列名称：")
        print(non_matching_columns['two_digits'])
        print("\n\n\n")
        print("四位数字形式的列名称：")
        print(non_matching_columns['four_digits'])
        print("\n\n\n")
        print("其他形式的列名称：")
        print(non_matching_columns['other'])
        print("\n\n\n")

        # 打印重命名的其他列名称
        print("重命名的其他列名称：")
        print(renamed_others)
        print("\n\n\n")

# 调用函数并传入CSV文件路径和JSON映射
filename = r'C:\Users\黄国斌\Desktop\模块化\temp\草地.csv'  # 替换成你的CSV文件路径
replace_and_print_column_names(filename, mapping)


poi_六位数字_dis的修改后列名称如下：
['距汽车用品、汽车养护的距离', '距汽车销售及服务的距离', '距乡、镇级政府机关的距离', '距商业性住宿的距离', '距超市的距离', '距陵园、公墓的距离', '距垂钓的距离', '距机场的距离', '距寺庙、道观的距离', '距宠物医院的距离', '距其他外国办事机构的距离', '距便利店的距离', '距院/系的距离', '距其他租赁服务的距离', '距其他服务的距离', '距行政办公大厅的距离', '距居民住宿的距离', '距社区医疗的距离', '距医药及医疗器材零售的距离', '距高尔夫练习场的距离', '距票务中心｜定票处的距离', '距宗教用品的距离', '距成人用品零售的距离', '距玩具店的距离', '距集镇的距离', '距花卉专卖店的距离', '距公共厕所的距离', '距学校报名处的距离', '距医疗器械的距离', '距福利院、敬老院的距离', '距超限超载检测站的距离', '距外国大使馆/领事馆的距离', '距体育用品店的距离', '距加油站的距离', '距省级地名的距离', '距住宅楼的距离', '距报社、出版社、杂志社的距离', '距星级宾馆的距离', '距茶叶、茶具专卖店的距离', '距婚姻介绍所的距离', '距旧货市场的距离', '距摩托车、自行车、电动车维护与保养的距离', '距母婴用品、儿童用品的距离', '距练歌房、KTV的距离', '距家装建材零售的距离', '距协会的距离', '距区县级地名的距离', '距桥的距离', '距音像店的距离', '距汽车维护与保养的距离', '距村屯级地名的距离', '距异国风味的距离', '距地方土特产专卖店的距离', '距电脑专卖店的距离', '距省/直辖市/自治区政府的距离', '距高速入口的距离', '距社会团体、协会的距离', '距教育的距离', '距互通的距离', '距街道级地名的距离', '距电信的距离', '距地上露天停车场的距离', '距工业园的距离', '距港口、码头的距离', '距溜冰场的距离', '距汽车及相关产品零售的距离', '距室内停车场的距离', '距母婴用品、儿童用品零售的距离', '距运动场馆的距离', '距公园的距离', '距羽毛球场的距离', '距培训中心的距离', '距牙科诊所的距离', '距其他单位的距

### 对csv中的列名称映射回中文名称

In [39]:
import csv
import json
import re

# 加载JSON文件
with open('all_mapping.json', 'r', encoding='utf-8') as f:
    mapping = json.load(f)

def replace_and_save_column_names(input_filename, output_filename, mapping):
    with open(input_filename, 'r', newline='', encoding='gbk') as csvfile:
        reader = csv.reader(csvfile)
        column_names = next(reader)

        # 使用正则表达式匹配各种格式的列名
        pattern_six_digits = re.compile(r'poi_(\d{6})_dis')
        pattern_two_digits = re.compile(r'poi_(\d{2})_dis')
        pattern_four_digits = re.compile(r'poi_(\d{4})_dis')
        pattern_generic_dis = re.compile(r'(.*)_dis$')

        new_column_names = []
        non_matching_columns = {
            'two_digits': [],
            'four_digits': [],
            'other': []
        }
        modified_six_digits = []
        modified_four_digits = []
        modified_two_digits = []
        modified_generic_dis = []
        renamed_others = []

        rename_mapping = {
            'RKMD': '人口密度',
            'DGSJ': '灯光数据',
            'QWSJ': '气温数据'
        }

        for col in column_names:
            match_six = pattern_six_digits.match(col)
            match_two = pattern_two_digits.match(col)
            match_four = pattern_four_digits.match(col)
            match_generic = pattern_generic_dis.match(col)
            
            if match_six:
                key = match_six.group(1)
                if key in mapping:
                    new_col_name = f"距{mapping[key]}的距离"
                    new_column_names.append(new_col_name)
                    modified_six_digits.append(new_col_name)
                else:
                    new_column_names.append(col)
                    non_matching_columns['other'].append(col)
            elif match_two:
                key = match_two.group(1)
                if key in mapping:
                    new_col_name = f"距{mapping[key]}的距离"
                    new_column_names.append(new_col_name)
                    modified_two_digits.append(new_col_name)
                else:
                    new_column_names.append(col)
                    non_matching_columns['other'].append(col)
            elif match_four:
                key = match_four.group(1)
                if key in mapping:
                    new_col_name = f"距{mapping[key]}的距离"
                    new_column_names.append(new_col_name)
                    modified_four_digits.append(new_col_name)
                else:
                    new_column_names.append(col)
                    non_matching_columns['other'].append(col)
            elif match_generic:
                new_col_name = f"距{match_generic.group(1)}的距离"
                new_column_names.append(new_col_name)
                modified_generic_dis.append(new_col_name)
            elif col in rename_mapping:
                new_col_name = rename_mapping[col]
                new_column_names.append(new_col_name)
                renamed_others.append(new_col_name)
            else:
                new_column_names.append(col)
                non_matching_columns['other'].append(col)
        
        # 打印修改后的列名称
        print("修改后所有的列名称：")
        print(new_column_names)
        print("\n\n\n")
        
        # 打印poi_六位数字_dis的修改列名称
        print("poi_六位数字_dis的修改列名称：")
        print(modified_six_digits)
        print("\n\n\n")
        
        # 打印poi_四位数字_dis的修改列名称
        print("poi_四位数字_dis的修改列名称：")
        print(modified_four_digits)
        print("\n\n\n")

        # 打印poi_两位数字_dis的修改列名称
        print("poi_两位数字_dis的修改列名称：")
        print(modified_two_digits)
        print("\n\n\n")

        # 打印形如xxx_dis的修改列名称
        print("形如xxx_dis的修改列名称：")
        print(modified_generic_dis)
        print("\n\n\n")
        
        # 打印不匹配的列名称
        print("其他形式的列名称：")
        print(non_matching_columns['other'])
        print("\n\n\n")

        # 打印重命名的其他列名称
        print("重命名的其他列名称：")
        print(renamed_others)
        print("\n\n\n")
        
        # 写回修改后的CSV文件
        with open(output_filename, 'w', newline='', encoding='utf-8') as csvfile:
            writer = csv.writer(csvfile)
            writer.writerow(new_column_names)
            for row in reader:
                writer.writerow(row)

# 调用函数并传入CSV文件路径和JSON映射
input_filename = r'C:\Users\黄国斌\Desktop\模块化\temp\工业用地.csv'  # 替换成你的CSV文件路径
output_filename = r'C:\Users\黄国斌\Desktop\模块化\导出修改列名\工业用地_修改后.csv'  # 替换成你希望保存的新CSV文件路径
replace_and_save_column_names(input_filename, output_filename, mapping)


修改后所有的列名称：
['pointID', 'Y', '距汽车用品、汽车养护的距离', '距汽车销售及服务的距离', '距乡、镇级政府机关的距离', '距商业性住宿的距离', '距超市的距离', '距陵园、公墓的距离', '距垂钓的距离', '距机场的距离', '距寺庙、道观的距离', '距汽车维护与保养的距离', '距旅游景区点的距离', '距宠物医院的距离', '距南宁市现状公园绿地（已建成）的距离', '距其他外国办事机构的距离', '距便利店的距离', '距定修段的距离', '距院/系的距离', '距其他租赁服务的距离', '距其他服务的距离', '距行政办公大厅的距离', '距居民住宿的距离', '距社区医疗的距离', '距饮料及烟酒制品零售的距离', '距社会团体、协会的距离', '距租赁服务的距离', '距现状110k伏埋地电缆的距离', '距供水路线的距离', '距医药及医疗器材零售的距离', '距高尔夫练习场的距离', '距票务中心｜定票处的距离', '距宗教用品的距离', '距厂矿企业的距离', '距餐饮的距离', '距公交场站的距离', '距成人用品零售的距离', '距玩具店的距离', '距集镇的距离', '距RD-02-快速路的距离', '距花卉专卖店的距离', '距运动、休闲的距离', '距公共厕所的距离', '距宗教的距离', '距学校报名处的距离', '距医疗器械的距离', '距福利院、敬老院的距离', '距餐馆的距离', '距超限超载检测站的距离', '距外国大使馆/领事馆的距离', '距交通附属设施的距离', '距普通站的距离', '距体育用品店的距离', '距加油站的距离', '距教育的距离', '距省级地名的距离', '距其他服务的距离', '距住宅楼的距离', '距报社、出版社、杂志社的距离', '距星级宾馆的距离', '距茶叶、茶具专卖店的距离', '距婚姻介绍所的距离', '距旧货市场的距离', '距摩托车、自行车、电动车维护与保养的距离', '距RD-04-公路的距离', '距母婴用品、儿童用品的距离', '距休闲度假的距离', '距练歌房、KTV的距离', '距家装建材零售的距离', '距三级路的距离', '距协会的距离', '距区县级地名的距离', '距家用电器及电子产品零售的距离', '距医药及医疗器材零售的距离

### 合并建设用地4类数据
- 在excel中手动添加了OBJECTID

In [44]:
import pandas as pd

# 定义文件路径和类别名称的映射
file_category_mapping = {
    './导出修改列名\工业用地_修改后.csv': '工业用地',
    './导出修改列名\住宅用地_修改后.csv': '住宅用地',
    './导出修改列名\商业用地_修改后.csv': '商业用地',
    './导出修改列名\公服用地_修改后.csv': '公服用地'
}
# 创建一个空的DataFrame用于存储合并后的数据
combined_df = pd.DataFrame()

# 遍历每个文件，读取数据，添加类别列，并合并到总的DataFrame中
for file, category in file_category_mapping.items():
    df = pd.read_csv(file)
    df['类别'] = category  # 添加类别列
    # 将类别列移动到第二列
    cols = df.columns.tolist()
    cols.insert(1, cols.pop(cols.index('类别')))
    df = df[cols]
    combined_df = pd.concat([combined_df, df], ignore_index=True)  # 合并数据

# 保存合并后的数据到新的CSV文件
combined_df.to_csv('建设用地_合并后.csv', index=False)

# 打印合并后的DataFrame的信息
print(combined_df.info())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6465 entries, 0 to 6464
Columns: 449 entries, pointID to 输气管道路线
dtypes: float64(442), int64(6), object(1)
memory usage: 22.1+ MB
None


### 合并农用地6类数据
- 在excel中手动添加了OBJECTID

In [45]:
import pandas as pd

# 定义文件路径和类别名称的映射
file_category_mapping = {
    './导出修改列名\草地_修改后.csv': '草地',
    './导出修改列名\园地_修改后.csv': '园地',
    './导出修改列名\耕地_修改后.csv': '耕地',
    './导出修改列名\林地_修改后.csv': '林地',
    './导出修改列名\设施农用地_修改后.csv': '设施农用地',
    './导出修改列名\坑塘水面_修改后.csv': '坑塘水面',

}
# 创建一个空的DataFrame用于存储合并后的数据
combined_df = pd.DataFrame()

# 遍历每个文件，读取数据，添加类别列，并合并到总的DataFrame中
for file, category in file_category_mapping.items():
    df = pd.read_csv(file)
    df['类别'] = category  # 添加类别列
    # 将类别列移动到第二列s
    cols = df.columns.tolist()
    cols.insert(1, cols.pop(cols.index('类别')))
    df = df[cols]
    combined_df = pd.concat([combined_df, df], ignore_index=True)  # 合并数据

# 保存合并后的数据到新的CSV文件
combined_df.to_csv('农用地_合并后.csv', index=False)

# 打印合并后的DataFrame的信息
print(combined_df.info())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5092 entries, 0 to 5091
Columns: 450 entries, pointID to 输气管道路线
dtypes: float64(442), int64(6), object(2)
memory usage: 17.5+ MB
None
